# QuTiP example: Vacuum Rabi oscillations in the Jaynes-Cummings model

This notebook is based on https://nbviewer.jupyter.org/github/qutip/qutip-notebooks/blob/master/examples/rabi-oscillations.ipynb#QuTiP-example:-Vacuum-Rabi-oscillations-in-the-Jaynes-Cummings-model
 
 This ipython notebook demonstrates how to simulate the quantum vacuum rabi oscillations in the Jaynes-Cumming model, using QuTiP: The Quantum Toolbox in Python.

 For more information about QuTiP see project web page: http://code.google.com/p/qutip/


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

# Introduction
 
 The Jaynes-Cumming model is the simplest possible model of quantum mechanical light-matter interaction, describing a single two-level atom interacting with a single electromagnetic cavity mode. The Hamiltonian for this system is (in dipole interaction form)
 
 $H = \hbar \omega_c a^\dagger a + \frac{1}{2}\hbar\omega_a\sigma_z + \hbar g(a^\dagger + a)(\sigma_- + \sigma_+)$
 
 or with the rotating-wave approximation
 
 $H_{\rm RWA} = \hbar \omega_c a^\dagger a + \frac{1}{2}\hbar\omega_a\sigma_z + \hbar g(a^\dagger\sigma_- + a\sigma_+)$
 
 where $\omega_c$ and $\omega_a$ are the frequencies of the cavity and atom, respectively, and $g$ is the interaction strength.
 
  Here we use units where $\hbar = 1$: 

In [41]:
def calculate(wc_per_2pi, wa_per_2pi, g_per_2pi, kappa, gamma, n_th_a, use_rwa = True):
    
    wc = wc_per_2pi  * 2 * np.pi  # cavity frequency
    wa = wa_per_2pi  * 2 * np.pi  # atom frequency
    g  = g_per_2pi * 2 * np.pi  # coupling strength
    #kappa = 0.005          # cavity dissipation rate
    #gamma = 0.05           # atom dissipation rate
    N = 15                 # number of cavity fock states
    #n_th_a = 0.0           # temperature in frequency units
    #use_rwa = True

    tlist = np.linspace(0,25,100)
    
    # intial state
    psi0 = tensor(basis(N,0), basis(2,1))    # start with an excited atom

    # operators
    a  = tensor(destroy(N), qeye(2))
    sm = tensor(qeye(N), destroy(2))

    # Hamiltonian
    if use_rwa:
        H = wc * a.dag() * a + wa * sm.dag() * sm + g * (a.dag() * sm + a * sm.dag())
    else:
        H = wc * a.dag() * a + wa * sm.dag() * sm + g * (a.dag() + a) * (sm + sm.dag())
    
    c_op_list = []

    rate = kappa * (1 + n_th_a)
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * a)

    rate = kappa * n_th_a
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * a.dag())

    rate = gamma
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * sm)
    
    output = mesolve(H, psi0, tlist, c_op_list, [a.dag() * a, sm.dag() * sm])
    
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(tlist, output.expect[0], label="Cavity")
    ax.plot(tlist, output.expect[1], label="Atom excited state")
    ax.legend()
    ax.set_xlabel('Time')
    ax.set_ylabel('Occupation probability')
    ax.set_title('Vacuum Rabi oscillations');

    
#calculate(wc,wa,g,kappa,gamma,n_th_a)

from ipywidgets import interact
interact(calculate, wc_per_2pi = (0, 2, 0.1), wa_per_2pi = (0, 2, 0.1), g_per_2pi = (0.00, 0.1, 0.01), 
         kappa = (0.0, 0.01, 0.001), gamma = (0.0, 0.10, 0.01), n_th_a = 0 )

interactive(children=(FloatSlider(value=1.0, description='wc_per_2pi', max=2.0), FloatSlider(value=1.0, descri…

<function __main__.calculate>